# 1. HuggingFaceH4/CodeAlpaca_20K Python split

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
from datasets import Dataset, DatasetDict, concatenate_datasets
from datasets import Features, Value
from datasets import  load_dataset, DownloadConfig
from datasets import load_from_disk
import pandas as pd

In [2]:
h4ca20k = load_dataset("HuggingFaceH4/CodeAlpaca_20K")

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(h4ca20k)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 18019
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2003
    })
})


In [11]:
print(h4ca20k["train"])

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 18019
})


In [61]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=open('./api/apikey/glmkeys.txt', 'r').read().strip()) 

In [23]:
def is_py(prompts):
    py_dataset = Dataset.from_dict({})
    for i in prompts:

        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
                {"role": "user", "content": i["prompt"]},
                
        
            ],
        )

        if(response.choices[0].message.content == "1"):
            py_dataset.add_item(i)

    return py_dataset


In [69]:
prompts  = h4ca20k["train"]

In [70]:

features = Features({
    'prompt': Value('string'),
    'completion': Value('string')
})


py_dataset = Dataset.from_dict({'prompt':[], 'completion':[]},features=features)

In [72]:

cot = 0
for i in prompts:

    response = client.chat.completions.create(
        model="glm-4", 
    
        messages=[


            {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
            {"role": "user", "content": i["prompt"]},
                
        
        ],
    )

    if (response.choices[0].message.content == "1"):
        
        
        if not(cot % 100):
            cot = cot+1
            print(cot)
            continue
        cot = cot+1

        py_dataset = concatenate_datasets([py_dataset,Dataset.from_dict({'prompt':[i['prompt']],'completion':[i['completion']]})])
    
    

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701


In [73]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 4777
})


In [62]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 548
})


In [63]:
py_dataset.save_to_disk('./data/codeNLU/h4ca20k_py/h4ca20k_py_test.parquet')

Saving the dataset (1/1 shards): 100%|██████████| 548/548 [00:00<00:00, 42153.81 examples/s]


In [58]:
py_test = py_dataset

In [74]:
py_train = py_dataset

In [75]:
py_train.save_to_disk('./data/codeNLU/h4ca20k_py/h4ca20k_py_train.parquet')

Saving the dataset (1/1 shards): 100%|██████████| 4777/4777 [00:00<00:00, 135963.94 examples/s]


In [68]:
h4ca20k_pytest = load_from_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py_test.parquet")

pytest_table = pa.Table.from_pandas(h4ca20k_pytest.to_pandas())

pq.write_table(pytest_table, './data/codeNLU/h4ca20k_py/h4ca20k_py_test_.parquet')



In [76]:
h4ca20k_pytrain = load_from_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py_train.parquet")

pytrain_table = pa.Table.from_pandas(h4ca20k_pytrain.to_pandas())

pq.write_table(pytrain_table, './data/codeNLU/h4ca20k_py/h4ca20k_py_train_.parquet')


In [79]:
dataset_dict = DatasetDict({
    'train': py_train,
    'test': py_test
})


dataset_dict.save_to_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py.parquet")


Saving the dataset (1/1 shards): 100%|██████████| 548/548 [00:00<00:00, 90953.21 examples/s]


In [80]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 4777
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 548
    })
})


In [81]:

train_df = dataset_dict['train'].to_pandas()
test_df = dataset_dict['test'].to_pandas()
train_df['partition'] = 'train'
test_df['partition'] = 'test'

combined_df = pd.concat([train_df, test_df])


combined_df.to_parquet('./data/codeNLU/h4ca20k_py/h4ca20k_py_.parquet')

In [83]:
dataset_dict.push_to_hub("graycatHCO3/CodeAlpaca-20K-Python")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/graycatHCO3/CodeAlpaca-20K-Python/commit/38048a8242c7b8745701cf970aa389c00308c5f5', commit_message='Upload dataset', commit_description='', oid='38048a8242c7b8745701cf970aa389c00308c5f5', pr_url=None, pr_revision=None, pr_num=None)

# 2. SFSA_e2t_0200 

In [27]:
import os 
import json
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from datasets import Dataset, DatasetDict, concatenate_datasets
from datasets import Features, Value
from datasets import  load_dataset, DownloadConfig
from collections import deque
import pickle
from datasets import load_from_disk
import pandas as pd
from src.utils.data_utils import format_data_sfsa

In [2]:

input_folder = './data/SFSA_0200/sfsa_0200_01'



In [3]:
agents = ['adversary_0(Thief)','agent_0(Elf Wizard)','agent_1(Human Warrior)']

In [4]:
sfsa_dict = {}
for file_name in os.listdir(input_folder):
    input_file_path = os.path.join(input_folder, file_name)

    with open(input_file_path, 'r') as file:
        data = json.load(file)

    formatted_data = format_data_sfsa(data,agents)
    dataset_name = file_name.replace('.json', '')
    dataset = Dataset.from_dict({'turn':[x['turn'] for x in formatted_data], 'details':[x['details'] for x in formatted_data]})
    
    sfsa_dict[dataset_name] = dataset

sfsa_dict = DatasetDict(sfsa_dict)
print(sfsa_dict)

DatasetDict({
    SFSA_0200_000001: Dataset({
        features: ['turn', 'details'],
        num_rows: 20
    })
    SFSA_0200_000002: Dataset({
        features: ['turn', 'details'],
        num_rows: 20
    })
    SFSA_0200_000003: Dataset({
        features: ['turn', 'details'],
        num_rows: 10
    })
    SFSA_0200_000004: Dataset({
        features: ['turn', 'details'],
        num_rows: 10
    })
    SFSA_0200_000005: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000006: Dataset({
        features: ['turn', 'details'],
        num_rows: 14
    })
    SFSA_0200_000007: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000008: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000009: Dataset({
        features: ['turn', 'details'],
        num_rows: 14
    })
    SFSA_0200_000010: Dataset({
        features: ['turn', 'details'],
        num_rows: 17
   

In [5]:
for i, game in enumerate(sfsa_dict["SFSA_0200_000002"]):
    print(game)
    for j in game:
        print(j)
    if i>2:
        break
    


{'turn': 'round0', 'details': {'actions': [{'adversary_0': 0, 'agent_0': 0, 'agent_1': 1}], 'next_state': {'adversary_0(Thief)': [-0.38, -0.04, -0.93, -0.05, -0.98, -1.65, -0.13, -1.07], 'agent_0(Elf Wizard)': [0.05, 1.6, 0.6, 1.61, 0.05, 1.6, 0.98, 1.65, 0.85, 0.58], 'agent_1(Human Warrior)': [-0.8, 1.02, -0.25, 1.03, -0.8, 1.02, 0.13, 1.07, -0.85, -0.58]}, 'reward': [{'adversary_0': -0.930179864627099, 'agent_0': -0.36342237418154866, 'agent_1': -0.36342237418154866}], 'state': [{'adversary_0(Thief)': [-0.38, -0.04, -0.93, -0.05, -0.98, -1.65, -0.13, -1.07], 'agent_0(Elf Wizard)': [0.05, 1.6, 0.6, 1.61, 0.05, 1.6, 0.98, 1.65, 0.85, 0.58], 'agent_1(Human Warrior)': [-0.8, 1.02, -0.25, 1.03, -0.8, 1.02, 0.13, 1.07, -0.85, -0.58]}]}}
turn
details
{'turn': 'round15', 'details': {'actions': [{'adversary_0': 0, 'agent_0': 0, 'agent_1': 4}], 'next_state': {'adversary_0(Thief)': [-0.38, -0.04, -0.93, -0.05, -0.98, -1.65, -0.18, -1.07], 'agent_0(Elf Wizard)': [0.05, 1.6, 0.6, 1.61, 0.05, 1.6,

In [ ]:
sfsa_dict.push_to_hub("graycatHCO3/Simple_SFSA_0200");

In [ ]:
sfsa_dict.save_to_disk("./data/HuggingFace/Simple_SFSA_0200");

In [4]:
sfsa = load_from_disk("./data/HuggingFace/Simple_SFSA_0200");

In [5]:
sfsa_e2t = sfsa

In [6]:
sfsa_e2t["SFSA_0200_000001"]

Dataset({
    features: ['turn', 'details'],
    num_rows: 20
})

In [7]:
sfsa_e2t["SFSA_0200_000001"][0]["turn"]

'round0'

In [8]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=open('./api/apikey/glmkeys.txt', 'r').read().strip())

In [9]:
with open('data\SFSA_e2t_0200\SFSA_e2t_0200_01\e2tSFSA_0200_000001.json', 'r') as file:
    test_sfsa = json.load(file)


In [10]:
index_map = {
    0 : 'adversary_0(Thief)',
    1 : 'agent_0(Elf Wizard)',
    2 : 'agent_1(Human Warrior)',
    3 : 'dialogues'
}

In [11]:
print(index_map[3])

dialogues


In [13]:

dialogue_que = deque(maxlen=2000)
for round, details in test_sfsa.items():
    cnt = np.zeros((10,10))
    cot = np.zeros(10)

    for speaker in details['order'][0]:
        speaker = int(speaker)
        #print(opt)
        opt = int(details['order'][speaker+1][int(cot[speaker])])
        cot[speaker] += 1
        #print('opt:',opt)
        #print(cnt[speaker][opt])
        #print('round:',round)        
        #print(details[index_map[speaker]][index_map[3]][opt])
        dialogue_que.append(

            f'{str(round)}:[{speaker}]{str(details[index_map[speaker]][index_map[3]][opt][int(cnt[speaker][opt])])}[{str(opt)}]'
            )
        cnt[speaker][opt] += 1



In [17]:
print(dialogue_que.pop())

199:[2]The strength of our arms and the courage of our hearts have brought us this far, reflected in the shifting numbers before us. Whatever awaits at the end, it is but a marker of the journey we've undertaken together. Let's finish this, side by side.[3]


In [19]:
dialogue_cnt = 1

In [28]:
with open(f'./data/SFSA_e2t_0200/SFSA_e2t_0200_01/dialogue_que/dialogue_que_{dialogue_cnt:06d}.pkl','wb') as f:
    pickle.dump(dialogue_que, f)

In [35]:
dialogue_cnt += 1

In [30]:

response = client.chat.completions.create(
    model="glm-4", 
    
    messages=[


        {"role": "system", "content": "The user will provide information from an episode in the reinforcement learning environment simple_adversary. The assistant needs to simulate appropriate dialogue between agents based on this information. Each line of dialogue should be prefixed with [0-2] to indicate the speaker, and suffixed with [0-3] to indicate the target. 0-3 represent adversary_0, agent_0, agent_1, and all, respectively. The dialogue should contain no more than five sentences."},
        {"role": "system", "content": "Example: 199:[2]The strength of our arms and the courage of our hearts have brought us this far[3]"},
        {"role": "user", "content": "Game: SFSA_0200_000002"},
        {"role": "user", "content": "round: "},
        {"role": "user", "content": sfsa_e2t["SFSA_0200_000002"][0]["turn"]},
        {"role": "user", "content": sfsa_e2t["SFSA_0200_000002"][0]["details"]},
        
    ],
)

print(response.choices[0].message.content)

[0] It seems my stealthy approach has been noticed. I must act quickly to turn this tide![3]

[1] Alert! The Thief is moving into position. We should coordinate our actions to counter their advance.[2]

[2] Roger that, Wizard. I'll distract the Thief while you prepare your spells![1]

[1] Understood, Warrior. I'll cast a spell to support you from the rear![0]

[0] You won't catch me off guard that easily! Prepare to face the consequences![3]


In [36]:
dataset_name=f'SFSA_0200_{dialogue_cnt:06d}'

In [50]:
dialogue_cnt -= 1


In [51]:
dialogue_cnt

5

In [52]:
for num in range(200):
    dialogue_que = deque()
    for game in sfsa_e2t[dataset_name]:
        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "The user will provide information from an episode in the reinforcement learning environment simple_adversary. The assistant needs to simulate appropriate dialogue between agents based on this information. Each line of dialogue should be prefixed with [0-2] to indicate the speaker, and suffixed with [0-3] to indicate the target. 0-3 represent adversary_0, agent_0, agent_1, and all, respectively. The dialogue should contain no more than five sentences."},
                {"role": "system", "content": "Example: 199:[2]The strength of our arms and the courage of our hearts have brought us this far[3]"},
                {"role": "user", "content": f"Game: {dataset_name}"},
                {"role": "user", "content": "round: "},
                {"role": "user", "content": game["turn"]},
                {"role": "user", "content": game["details"]},
        
            ],
        )
        results = response.choices[0].message.content.splitlines()
        for i in results:
            if i:
                dialogue_que.append(game['turn'] + ":" + i)
    print(dialogue_que.pop())
    dialogue_cnt = dialogue_cnt + 1
    print(dialogue_cnt)
    with open(f'./data/SFSA_e2t_0200/SFSA_e2t_0200_01/dialogue_que/dialogue_que_{dialogue_cnt:06d}.pkl','wb') as f:
        pickle.dump(dialogue_que, f)



round199:[2] Together we can overcome the Adversary's tactics. Let's press on with a united front![all]
6
round199:[1] Let's coordinate our moves. We can't afford to let our guard down![3]
7
round199:[1] Don't let the Thief's movements distract you, we need to coordinate our efforts![3]
8
round199:[2] I'll guard the Wizard. You focus on the Thief, and let's take him down![1]
9
round199:[1] Don't underestimate our combined strength. We must work together to protect our goal![0]
10
round199:[2] I will focus my attacks and try to improve our collective outcome. We must not let the Adversary's actions bring us down![3]
11
round199:[1] Be cautious, the Thief is known for their deception. Stick together![2]
12
round199:[1] Be careful, everyone. The Thief is desperate and may do something unpredictable![3]
13
round199:[1] Be cautious, everyone. The Thief is known for its deception![3]
14
round199:[1] Be cautious, friends. The Thief may have some surprises for us![3]
15
round199:[1] Be cautiou

In [48]:
with open(f'./data/SFSA_e2t_0200/SFSA_e2t_0200_01/dialogue_que/dialogue_que_000005.pkl','rb') as f:
    load_dialogue = pickle.load(f)
print(load_dialogue)


deque(["round0:[0] It seems I've been caught off guard![3]", "round0:[1] Don't worry, we're here to help![0]", "round0:[2] Let's surround the thief and take him down together![3]", "round0:[0] I'll use my agility toFlank from the side![1]", "round0:[1] I'll cast a spell to support![2]", 'round15:[0] It seems my stealthy approach has been noticed. I must resort to other means[3]', 'round15:[1] Be wary, my allies! The thief is desperate and may strike soon[3]', "round15:[2] I'll ready my blade. Let's not give the adversary an opening[3]", 'round16:[0] It seems my stealthy approach has been noticed. I must refine my tactics.[3]', 'round16:[1] The Thief is trying to be sly, but I can sense their movements. We should coordinate our attacks to corner them![2]', "round16:[2] I agree, agent_0. Let's use the element of surprise and strike from different angles![0]", "round16:[0] You won't catch me off guard so easily. I'm adapting to your strategies![3]", "round16:[1] We'll see about that. Agen